# Example Socket Client: Addition calculator

We will implement an extremely simple calculator. See the `example_socket_server.ipynb` notebook for the server! **Most of the explanation and documentation is in the server.**

For convenience, we repeat the steps here:

| Step | Server | Client |
| -  | - | - |
|  1 | Create server socket with `socket()` |  |
|  2 | Bind server socket to address with `bind()` |  |
|  3 | Enter listening state with `listen()` | Create client socket with `socket()` |
|  4 |  | Connect to server with `connect()` |
|  5 | Accept connection with `accept()` | |
|  6 | | Send integer with `send` e.g. `7`| 
|  7 | Receive integer with `recv` | |
|  8 | | Send integer with `send` e.g. `10`| 
|  9 | Receive integer with `recv` | |
| 10 | Calculate `10 + 7 = 17` | |
| 11 | Send integer with `send` (e.g. `17`) | |
| 12 | | Receive integer with `recv` |
| 13 | Quit | Quit |

---

## Step 3: Create client socket with `socket()`

Only run this step *after* you have run Step 0, 1, and 2 on the server.

In [1]:
import socket
import struct

ADDRESS_FAMILY = socket.AF_UNIX
SOCKET_TYPE = socket.SOCK_STREAM
ADDRESS = './some_filename.sock'

clientsocket = socket.socket(ADDRESS_FAMILY, SOCKET_TYPE)

---

## Step 4: Connect to server with `connect()`

Only run this *after* you have run at least step 3 on the server.


> **"Do I need to worry about timing?"** Not really, just run your server code first! Once the server has run `bind` and `listen`, it will wait on `accept` until the client connects with `connect`.

(The order doesn't actually matter, so long as the server runs first.)

In [2]:
clientsocket.connect(ADDRESS)

---

## Main part: Sending and receiving data

Finally, we move to actual data transfering! :) Let's see how this goes.

### Step 6: Send integer with `send`

Here, we prepare an integer (7) that is 8-bytes-long with little-endian encoding.
 * We use 8 bytes so we can prepare data as `long long` integers, just as an example :)
 * "Little-endian" is the bit-order, and so long as we keep this consistent,

The server will use `recv` to read the bytes we send. First, we have to pack them to bytes, using `struct`. See the `struct` documentation here: https://docs.python.org/3/library/struct.html

In [3]:
# Here, we pack the integer '7' into the `long long` format ('q')
# with little-endian encoding ('<').
x = struct.pack('<q', 7)

print(x)
clientsocket.send(x)

# When we send data, it will print out the number of bytes sent (in this case, 8.)

b'\x07\x00\x00\x00\x00\x00\x00\x00'


8

---

### Step 8: Send integer with `send`

Now, we send another integer, `10` to the socket! It's just the same as above.

Note that we see `\n` as the first integer. This is because the 10th character of ASCII is the newline character, represnted as `\n`.

In [4]:
y = struct.pack('<q', 10)
print(y)
clientsocket.send(y)

b'\n\x00\x00\x00\x00\x00\x00\x00'


8

---

### Step 12: Receive integer with `recv`


Although sockets require you pack and unpack your bytes, the natural blocking mechanism (wherein `accept` and `recv` must wait) solves so many nightmare issues of race-conditions and deadlock that would otherwise need to be solved by implementing complex algorithms.

In [5]:
result_from_server = clientsocket.recv(8)
print(result_from_server)

b'\x11\x00\x00\x00\x00\x00\x00\x00'


In [6]:
result_from_server = struct.unpack('<q', result_from_server)[0]
print(result_from_server)

17


### Step 13: The end! We close the connection

In [7]:
clientsocket.close()